In [2]:
import os
import numpy as np
import PIL
from PIL import Image

import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

import random
from torchvision.transforms import Compose, RandomCrop, ToTensor, ToPILImage, CenterCrop, Resize
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
#Intall Kaggle API and create kaggle directory
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir .kaggle
#This data is used to login  into your Kaggle account
import json
token = {"username":"phoebeli9797","key":"de1e943d273136e775a9165770f5b5a1"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

     |████████████████████████████████| 58 kB 4.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=b19d6ed5627e300b27247b79f1dd0f2d198c2bb89fc6f14106de8f419d545e17
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [4]:
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json /root/.kaggle/
!kaggle config set -n path -v /content

- path is now set to: /content


In [5]:
!kaggle competitions download -c idl-fall21-hw2p2s1-face-classification
!pwd

 98% 855M/876M [00:04<00:00, 205MB/s]
100% 876M/876M [00:04<00:00, 196MB/s]
/content


In [6]:
!unzip competitions/idl-fall21-hw2p2s1-face-classification/idl-fall21-hw2p2s1-face-classification.zip

Streaming output truncated to the last 5000 lines.
  inflating: val_data/2348/4305.jpg  
  inflating: val_data/2348/505.jpg   
  inflating: val_data/2349/696.jpg   
  inflating: val_data/2349/736.jpg   
  inflating: val_data/235/7977.jpg   
  inflating: val_data/235/7990.jpg   
  inflating: val_data/2350/3052.jpg  
  inflating: val_data/2350/96.jpg    
  inflating: val_data/2351/5845.jpg  
  inflating: val_data/2351/7551.jpg  
  inflating: val_data/2352/1061.jpg  
  inflating: val_data/2352/2172.jpg  
  inflating: val_data/2353/1648.jpg  
  inflating: val_data/2353/6768.jpg  
  inflating: val_data/2354/481.jpg   
  inflating: val_data/2354/4832.jpg  
  inflating: val_data/2355/3324.jpg  
  inflating: val_data/2355/49.jpg    
  inflating: val_data/2356/1101.jpg  
  inflating: val_data/2356/3114.jpg  
  inflating: val_data/2357/2597.jpg  
  inflating: val_data/2357/3793.jpg  
  inflating: val_data/2358/242.jpg   
  inflating: val_data/2358/424.jpg   
  inflating: val_data/2359/6679.jpg  

In [ ]:
### Set up 
#### reference articals and documentation. 
####  Documentation: https://pytorch.org/vision/stable/models.html
#### https://zhuanlan.zhihu.com/p/149649900 (this page is talking about torchvision transforms' method)
#### ResNet and VGG reference https://github.com/bubbliiiing/classification-pytorch
### resnet.py 

In [3]:
normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomRotation(degrees = 45),
    torchvision.transforms.RandomCrop(64),
    torchvision.transforms.ColorJitter(hue = 0.2, saturation = 0.2),
    torchvision.transforms.RandomHorizontalFlip(p = 0.65),
    torchvision.transforms.RandomVerticalFlip(p = 0.45),
    torchvision.transforms.ToTensor(),
    normalize
])
train_dataset = torchvision.datasets.ImageFolder(root='train_data', 
                                                 transform = transforms)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, 
                                               shuffle=True, num_workers=2)

In [4]:

val_transforms = torchvision.transforms.Compose([
                                             torchvision.transforms.ToTensor(),
                                             normalize
])

val_dataset = torchvision.datasets.ImageFolder(root='val_data', 
                                                 transform= val_transforms)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, 
                                               shuffle=False, num_workers=2)

In [5]:
print(train_dataset.__len__(), len(train_dataset.classes))
print(val_dataset.__len__(), len(val_dataset.classes))

380638 4000
8000 4000


In [6]:
# This is the simplest possible residual block, with only one CNN layer.
# Looking at the paper, you can extend this block to have more layers, bottleneck, grouped convs (from shufflenet), etc.
# Or even look at more recent papers like resnext, regnet, resnest, senet, etc.
#Conv -> ReLu _ > Conv -> ReLU
#Plan to try, ResNet 50, Probably using VGG. Refernece Github, video, and Paper: https://github.com/bubbliiiing/classification-pytorch

class SimpleResidualBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channel, out_channel, stride=1, downsample=None, **kwargs):
        super(SimpleResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel,
                               kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out

In [7]:
#only for res50
#only for res50, res34 use simpleResidualBlock
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channel, out_channel, stride=1, downsample=None,
                 groups=1, width_per_group=64):
        super(Bottleneck, self).__init__()

        width = int(out_channel * (width_per_group / 64.)) * groups

        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=width,
                               kernel_size=1, stride=1, bias=False)  # squeeze channels
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2 = nn.Conv2d(in_channels=width, out_channels=width, groups=groups,
                               kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(width)
        self.conv3 = nn.Conv2d(in_channels=width, out_channels=out_channel*self.expansion,
                               kernel_size=1, stride=1, bias=False)  # unsqueeze channels
        self.bn3 = nn.BatchNorm2d(out_channel*self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out += identity
        out = self.relu(out)

        return out

In [8]:
#reference, resnet.py
class ResNet(nn.Module):
    def __init__(self,
                 block,
                 blocks_num,
                 num_classes=1000,
                 include_top=True,
                 groups=1,
                 width_per_group=64):
        super(ResNet, self).__init__()
        self.include_top = include_top
        self.in_channel = 64

        self.groups = groups
        self.width_per_group = width_per_group

        self.conv1 = nn.Conv2d(3, self.in_channel, kernel_size=3, stride=2,
                               padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channel)
        self.relu = nn.ReLU(inplace=True)
        #self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, blocks_num[0])
        self.layer2 = self._make_layer(block, 128, blocks_num[1], stride=2)
        self.layer3 = self._make_layer(block, 256, blocks_num[2], stride=2)
        self.layer4 = self._make_layer(block, 512, blocks_num[3], stride=2)
        if self.include_top:
            self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # output size = (1, 1)
            self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    def _make_layer(self, block, channel, block_num, stride=1):
        downsample = None
        if stride != 1 or self.in_channel != channel * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channel, channel * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(channel * block.expansion))

        layers = []
        layers.append(block(self.in_channel,
                            channel,
                            downsample=downsample,
                            stride=stride,
                            groups=self.groups,
                            width_per_group=self.width_per_group))
        self.in_channel = channel * block.expansion

        for _ in range(1, block_num):
            layers.append(block(self.in_channel,
                                channel,
                                groups=self.groups,
                                width_per_group=self.width_per_group))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
       # x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        if self.include_top:
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
            x = self.fc(x)

        return x


In [9]:
#for the resnet, only use 34 and 50. Ignore 18 due to the low performance
def resnet18(num_classes = 1000, include_top = True):
  return ResNet(SimpleResidualBlock, [2, 2, 2, 2], num_classes = num_classes, include_top = include_top)
def resnet34(num_classes=1000, include_top=True):
    return ResNet(SimpleResidualBlock, [3, 4, 6, 3], num_classes=num_classes, include_top=include_top)

def resnet50(num_classes=1000, include_top=True):
    return ResNet(Bottleneck, [3,8,36,3], num_classes=num_classes, include_top=include_top)
def resnet101(num_classes=1000, include_top=True):
  
    return ResNet(Bottleneck, [3, 4, 23, 3], num_classes=num_classes, include_top=include_top)

In [10]:
#plain layers
def train_model(epoch = 20, in_features = 3, lr = 5e-2, wd = 5e-5):
  numEpochs = epoch
  in_features = in_features # RGB channels

  learningRate = lr
  weightDecay = wd

  num_classes = len(train_dataset.classes)

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  network = ClassificationNetwork(in_features, num_classes)
  network = network.to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
  for epoch in range(numEpochs):
    
    # Train
    network.train()
    avg_loss = 0.0
    for batch_num, (x, y) in enumerate(train_dataloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = network(x)

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()
        if batch_num % 10 == 9:
        #    print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, avg_loss/50))
            avg_loss = 0.0
    
    # Validate
    network.eval()
    num_correct = 0
    for batch_num, (x, y) in enumerate(val_dataloader):
        x, y = x.to(device), y.to(device)
        outputs = network(x)
        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
        
    print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch, num_correct / len(val_dataset)))


In [ ]:
num_classes = len(train_dataset.classes)

#model
network_18 = resnet18(num_classes, include_top=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network_18 = network_18.to(device)

#hyperparameter
numEpochs = 25
in_features = 3 #RGB channels
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network_18.parameters(), lr=0.01, weight_decay=5e-5, momentum=0.9)


for epoch in range(numEpochs):
    
    # Train
    network_18.train()
    avg_loss = 0.0
    for batch_num, (x, y) in enumerate(train_dataloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = network_18(x)

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()

        if batch_num % 100 == 99:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, avg_loss/50))
            avg_loss = 0.0
    
    # Validate
    network_18.eval()
    num_correct = 0
    for batch_num, (x, y) in enumerate(val_dataloader):
        x, y = x.to(device), y.to(device)
        outputs = network_18(x)
        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
        
    print('Epoch: {}, Validation Accuracy: {:.2f}'.format(epoch, num_correct / len(val_dataloader)))

Epoch: 0	Batch: 100	Avg-Loss: 16.7254
Epoch: 0	Batch: 200	Avg-Loss: 16.6103
Epoch: 0	Batch: 300	Avg-Loss: 16.5694
Epoch: 0	Batch: 400	Avg-Loss: 16.5371
Epoch: 0	Batch: 500	Avg-Loss: 16.5081
Epoch: 0	Batch: 600	Avg-Loss: 16.4690
Epoch: 0	Batch: 700	Avg-Loss: 16.4235
Epoch: 0	Batch: 800	Avg-Loss: 16.3536
Epoch: 0	Batch: 900	Avg-Loss: 16.2987
Epoch: 0	Batch: 1000	Avg-Loss: 16.2498
Epoch: 0	Batch: 1100	Avg-Loss: 16.1708
Epoch: 0	Batch: 1200	Avg-Loss: 16.1698
Epoch: 0	Batch: 1300	Avg-Loss: 16.0991
Epoch: 0	Batch: 1400	Avg-Loss: 16.0748
Epoch: 0	Batch: 1500	Avg-Loss: 16.0098
Epoch: 0	Batch: 1600	Avg-Loss: 15.9907
Epoch: 0	Batch: 1700	Avg-Loss: 15.9598
Epoch: 0	Batch: 1800	Avg-Loss: 15.8950
Epoch: 0	Batch: 1900	Avg-Loss: 15.8355
Epoch: 0	Batch: 2000	Avg-Loss: 15.8185
Epoch: 0	Batch: 2100	Avg-Loss: 15.7442
Epoch: 0	Batch: 2200	Avg-Loss: 15.7464
Epoch: 0	Batch: 2300	Avg-Loss: 15.6273
Epoch: 0	Batch: 2400	Avg-Loss: 15.5649
Epoch: 0	Batch: 2500	Avg-Loss: 15.4897
Epoch: 0	Batch: 2600	Avg-Loss: 15.

In [ ]:
#pick up the best model

In [16]:
#test Loader
class ImageDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list
        self.img = [f for f in os.listdir(self.file_list) if os.path.isfile(os.path.join(self.file_list, f))]

    def __len__(self):
        return len(self.img)

    def __getitem__(self, index):
        filename = self.file_list + '/'+ self.img[index]
        img = Image.open(filename)
        img = torchvision.transforms.ToTensor()(img)
        img = normalize(img)
        return img,self.img[index]

test_data = ImageDataset('test_data')
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, 
                                               shuffle=False, num_workers=2)

In [17]:

network_18.eval()
preds_total = []
names = []
with torch.no_grad():
  for X_test, name in test_loader:
   # print(name)
    names.append(name)
    X_test = X_test.to(device='cuda')
    y_hat = network_18(X_test)
    
    y_hat = y_hat.cpu()
    y_hat = y_hat.detach().numpy()
    preds = np.argmax(y_hat, axis = 1)
    preds = preds.reshape(-1,1)
    preds_total.append(preds)

In [18]:
new_idx = train_dataset.class_to_idx
new_idx.keys()
new_idx.values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 21

In [19]:

finalnames = []
for k in names:
  #print(k)

  for n in range(0,len(k)):
    #print(k[n])

    finalnames.append(k[n])

In [20]:
preds_final = []
for i in preds_total:
  for t in i:
    
    add = [key for key, val in new_idx.items() if val == t]
    #print(add[0], t)
    preds_final.append(int(add[0]))
len(preds_final)

8000

In [21]:
import pandas as pd
df = pd.DataFrame({'id':finalnames, "label":preds_final})
df.to_csv("predict_9.csv", index=False)

In [22]:
from google.colab import files
files.download("predict_9.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
torch.save(network_18.state_dict(),'/content/gdrive/MyDrive/test')

In [23]:
!kaggle competitions download -c idl-fall21-hw2p2s2-face-verification

 91% 142M/157M [00:00<00:00, 215MB/s]
100% 157M/157M [00:00<00:00, 219MB/s]


In [24]:
!unzip /content/competitions/idl-fall21-hw2p2s2-face-verification/idl-fall21-hw2p2s2-face-verification.zip

Streaming output truncated to the last 5000 lines.
  inflating: verification_data/verification_data/00064100.jpg  
  inflating: verification_data/verification_data/00064101.jpg  
  inflating: verification_data/verification_data/00064102.jpg  
  inflating: verification_data/verification_data/00064103.jpg  
  inflating: verification_data/verification_data/00064104.jpg  
  inflating: verification_data/verification_data/00064105.jpg  
  inflating: verification_data/verification_data/00064106.jpg  
  inflating: verification_data/verification_data/00064107.jpg  
  inflating: verification_data/verification_data/00064108.jpg  
  inflating: verification_data/verification_data/00064109.jpg  
  inflating: verification_data/verification_data/00064110.jpg  
  inflating: verification_data/verification_data/00064111.jpg  
  inflating: verification_data/verification_data/00064112.jpg  
  inflating: verification_data/verification_data/00064113.jpg  
  inflating: verification_data/verification_data/0006

In [27]:
test_pair = np.genfromtxt('verification_pairs_test.txt',dtype='str')

new_pair = []
for i in test_pair: 
  pair = []
  for f in i:
    pair.append('/content/verification_data/' + f)
  new_pair.append(pair)
new_arr = np.array(new_pair, dtype='str')
new_arr[0][1]

'/content/verification_data/verification_data/00035322.jpg'

In [28]:
class Test_Dataset(Dataset):
    def __init__(self, file_list_1,file_list_2):
        self.file_list_1 = file_list_1
        self.file_list_2 = file_list_2

    def __len__(self):
        return len(self.file_list_1)

    def __getitem__(self, index):
        img_1 = Image.open(self.file_list_1[index])
        img_2 = Image.open(self.file_list_2[index])
        img_1 = torchvision.transforms.ToTensor()(img_1)
        img_1 = torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(img_1)
        img_2 = torchvision.transforms.ToTensor()(img_2)
        img_2 = torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(img_2)
        return img_1,img_2
test_dataset = Test_Dataset(new_arr[:,0],new_arr[:,1])
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                        shuffle=False,
                        num_workers=8,
                        batch_size=128,pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [29]:
from tqdm import tqdm
network_18.eval()
similarity_list = []
compute_sim = nn.CosineSimilarity(dim=1)
with torch.no_grad():
    for batch_num, (img_1, img_2) in tqdm(enumerate(test_dataloader),position=0, leave=True):
        img_1, img_2 = img_1.to(device), img_2.to(device)
        embedding_out1= network_18(img_1)
        embedding_out2= network_18(img_2)
        
        outputs = compute_sim(embedding_out1, embedding_out2).cpu().detach().numpy()
        outputs = outputs.reshape((len(outputs), 1))
        similarity_list.append(outputs)


similarity_list = np.vstack(similarity_list)
similarity_list.astype(np.float32)
similarity_list = np.reshape(similarity_list, (len(similarity_list),))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
405it [00:43,  9.36it/s]


In [30]:
ID_name = [elem for elem in test_pair] 
similarity_list_result = similarity_list.reshape(len(similarity_list))
similarity_list_result = [elem for elem in similarity_list_result]
  

In [31]:
with open("similarity4.csv", 'w') as fh:
  fh.write('id,category\n') 
  for i in range(len(similarity_list_result)):
    fh.write(ID_name[i][0]+' '+ ID_name[i][1]+ ',' + str(similarity_list_result[i]) + "\n")

In [32]:
from google.colab import files
files.download("similarity4.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>